In [2]:
import os
import glob
import pandas as pd
import re

In [17]:
conjunto_dados = pd.DataFrame()

speech_by_speaker = {}

files_checked = 0

# for file in os.scandir("./discursos"):
for file in os.scandir("./discursos"):
    files_checked += 1
    if (files_checked > 1):
        continue
    with open(file) as fp:
        content = fp.read()

        #1) remover datas espalhadas, páginas e afins espalhadas pelos documentos (por causa da coversao PDF para TXT)

        datas_texto = re.compile("\d{1,2} \w+ \w+ \w+ \d{4}")
        datas_texto.sub(" ", content)

        series_numeros = re.compile("[A-Z]+ SÉRIE \W NÚMERO \d+")
        series_numeros.sub(" ", content)

        #2) criar dicionario de intervencoes para criar dataframe

        #2.1) identificar os intervenientes e intervencoes correspondentes

        speaker_regex = re.compile(r'^[A|O] Sr\.ª?( [\w\-]+)+([\-]\w+)?( \(\w+\))*(?=:)', flags=re.M) 
        deputadxs = speaker_regex.finditer(content) 
        deputadxs = [match.group() for match in deputadxs]

        # print(deputadxs)

        content_on_one_line = ''.join(content.splitlines())

        deputadxs = [deputadxs[0]]

        for speaker in deputadxs:
            content_split_by_speaker = content_on_one_line.split(speaker + ':')
            # Get rid of everything before they speak
            content_split_by_speaker.pop(0)

            # what_they_say = [speech for speech in speaker_regex]
            # for each line in content_split_by_speaker
            # use the speaker regex to find the next speaker and get everything before that
            for line in content_split_by_speaker:
                regex = re.compile(r'[A|O] Sr\.ª?( [\w\-]+)+([\-]\w+)?( \(\w+\))*(?=:)')
                start_of_next_speaker = regex.finditer(line)
                print([match.start() for match in start_of_next_speaker])
                # either an index or None
                next_speaker_index = [match.start() for match in start_of_next_speaker][0]


            

        '''''
        conjunto_raw = pd.DataFrame()

        conjunto_raw['filename'] = os.path.basename(file)
        conjunto_raw["dia"] = dia
        conjunto_raw["ano"] = ano

        print(conjunto_raw)


        conjunto_dados = pd.concat([conjunto_dados, conjunto_raw])'''''
    

[]
[209, 1336, 1377, 2607, 2648, 3562, 3631, 4164, 4205]
[106]
[81]
[86]
[103, 931, 968, 1347, 1377]
[81]
[34]
[94, 1487, 1528, 1635, 1683, 2212, 2253]
[1667]
[]
[]
[126]
[81]
[73]
[95, 465, 644, 685, 893, 1041, 1082, 1458, 1499, 1687]
[]
[]
[80, 1515, 1552]
[40]
[75, 441, 486]
[34]
[]
[81]
[41]
[84]
[34]
[582]
[]
[104, 3699]
[87]
[191, 796]
[132]
[122]
[50]
[39, 341]
[114, 1448, 1496, 2443]
[102, 255, 299]
[115, 835, 871, 919, 1412, 1476, 2331]
[41]
[1929]
[]
[82, 1759, 1803]
[34]
[34]
[65]
[]
[]
[]
[105, 1876, 1917, 2884, 2933]
[34, 178]
[75]
[122]
[]
[]
[]
[70, 1067, 1139, 1935, 1977, 2359, 2399]
[41]
[62, 3201, 4327, 4372]
[81, 355, 402, 2516, 3065, 3356, 3455, 4111, 4151, 5927, 7053, 7098, 8444, 8792, 8846, 9192, 9244, 10153, 10721, 10882, 11224, 12145, 12188, 14695, 14740, 16505, 16550, 17292, 17458, 18699, 19215, 21138, 23825, 25353]
[2954, 5370, 9787, 13915, 14599, 14639, 15208, 15248, 15966, 16014, 16770, 17558, 18580, 22715, 23289, 25899, 26389, 26770, 28720, 29504, 31336, 31